## Trees: Ensemble Methods - Boosting

Boosting is another ensemble technique to create a collection of predictors. In this technique, learners are learned sequentially with early learners fitting simple models to the data and then analyzing data for errors. In other words, we fit consecutive trees (random sample) and at every step, the goal is to solve for net error from the prior tree.

When an input is misclassified by a hypothesis, its weight is increased so that next hypothesis is more likely to classify it correctly. By combining the whole set at the end converts weak learners into better performing model.

An ensemble of trees are built one by one and individual trees are summed sequentially. The Next tree tries to recover the loss (difference between actual and predicted values) from the previous tree.

 - boosting = low variance, high bias base learners



#### Adaboost = Adaptive Boosting
AdaBoost learns from the mistakes by increasing the weight of misclassified data points.

Steps:
<li> 0: Initialize the weights of data points. (e.g. data has 1000 points, each initial point would have 1/1000 = 0.001) </li>
<li> 1: Train a decision Tree (whole dataset) </li>
<li> 2: Calculate the weighted error rate (e) of the decision tree. </li>
<li> 3: Calculate this decision tree’s weight in the ensemble the weight of this tree = learning rate * log( (1 — e) / e) </li> 
<br> ** The higher the weighted error of the tree, the less decision power the tree will be given during the later voting. </br>
<br> ** The lower the weighted error of the tree, the higher decision power the tree will be given during the later voting. </br>

<li> 4: Update weights of wrongly classified points. </li> 
<br> the weight of each data point stays same if the model got this data points correct.</br> 
<br> the new weight of this data point = old weight* np.exp(weight of the tree), if the model got this data point wrong </br> 

<li> 5: Repeat step 1 (dataset with new weights) </li>
<li> 6: Make final prediction </li>

#### Gradient Boosting = Gradient Descent + Boosting.


Steps:
<li> 1. Train a decison Tree </li> 
<li> 2. Apply the decision Tree just trained to predict. </li> 
<li> 3. Calculate the residual of this decision tree, save residual errors as new y. </li> 
<li> 4. Repeat step 1 (until no of trees set to train is attained). </li> 
<li> 5. Make the final prediction. </li> 

<strong>Note:</strong>

<li> Gradient Boosting is prone to Over-fitting.</li>
<li> Requires careful tuning of different hyper-parameters.</li>

In [21]:
#import libraries
import xgboost as xgb
from sklearn.datasets import load_boston
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
import time
import catboost as cb

In [22]:
#import dataset

X,y = load_boston(return_X_y=True)

#train,test split
X_train,X_test,y_train,y_test = train_test_split(X,y,test_size=0.2,random_state=42)

#xgboost
xgbr = xgb.XGBRegressor(max_depth=5,learning_rate=0.1,n_estimators=100,n_jobs=1)
start_time = time.time()

xgbr.fit(X_train,y_train)


y_predict = xgbr.predict(X_test)

print("--- %s seconds ---" % (time.time() - start_time)) 

mean_squared_error(y_test,y_predict) #error

[01:19:33] WARNING: src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
--- 0.061842918395996094 seconds ---


6.583590106471756

In [24]:
#catboost helps you savetime on preprocessing for categorical columns for you. It is also reportedly faster.

#lets try catboost
cbr = cb.CatBoostRegressor(learning_rate=0.1,n_estimators=100,max_depth=5)

start_time = time.time()

cbr.fit(X_train,y_train)

y_predict = cbr.predict(X_test)

print("--- %s seconds ---" % (time.time() - start_time))

mean_squared_error(y_test,y_predict)    #error

0:	learn: 8.7268017	total: 1.14ms	remaining: 113ms
1:	learn: 8.2099159	total: 2.14ms	remaining: 105ms
2:	learn: 7.7968101	total: 3.11ms	remaining: 100ms
3:	learn: 7.3306938	total: 4.09ms	remaining: 98.2ms
4:	learn: 6.9727966	total: 4.96ms	remaining: 94.2ms
5:	learn: 6.6423222	total: 5.82ms	remaining: 91.2ms
6:	learn: 6.3111468	total: 6.97ms	remaining: 92.7ms
7:	learn: 6.0132589	total: 7.96ms	remaining: 91.5ms
8:	learn: 5.7434365	total: 8.98ms	remaining: 90.9ms
9:	learn: 5.5069788	total: 9.91ms	remaining: 89.2ms
10:	learn: 5.2635150	total: 11.4ms	remaining: 92ms
11:	learn: 4.9977830	total: 12.8ms	remaining: 93.7ms
12:	learn: 4.8349713	total: 14.1ms	remaining: 94.2ms
13:	learn: 4.6642290	total: 15ms	remaining: 92.3ms
14:	learn: 4.5238923	total: 15.9ms	remaining: 90.2ms
15:	learn: 4.3504734	total: 16.8ms	remaining: 88.2ms
16:	learn: 4.1976125	total: 17.6ms	remaining: 86.1ms
17:	learn: 4.1008217	total: 18.5ms	remaining: 84.4ms
18:	learn: 3.9720598	total: 19.6ms	remaining: 83.7ms
19:	learn:

9.344821856482579

Exercise: Load the intercampusai2019 dataset, train a random forest, extra trees model on the dataset and compare results using both random forests and gradient boosting under the *Gini and Entropy* criterions. 

<strong>Note: Also make sure to do some data cleaning, upsampling/downsampling, parameter tuning.</strong>

`n_estimators`
- increasing num trees wont affect bias, will only reduce variance

`max_features`
- how many features to split on
- rule of thumb = sqrt(num_features)
- depends on ratio of noisy to important var in dataset
- small num features = reduce variance increase bias
- lots of noisy = small m will decrease probability of choosing an important variable at a split

`min samples per leaf` 
- increase a bit (default is 1) to get smaller trees w less overfitting

`max_depth`
- controls variance

`subsample`
- The fraction of observations to be selected for each tree. Selection is done by random sampling.
- Values slightly less than 1 make the model robust by reducing the variance.



## Starting point hyperparameters

*** Heard from a Kaggle Grandmaster

Learning rate = 0.05, 1000 rounds, max depth = 3-5, subsample = 0.8-1.0, colsample_bytree = 0.3 - 0.8, lambda = 0 to 5

Add capacity to combat bias - add rounds

Reduce capacity to combat variance - depth / regularization